# PROYECTO – ENTREGA #3

Universidad de los Andes  
Ingeniería de Sistemas y Computación  
Modelado, Simulación y Optimización

* Esteban Castelblanco Gómez – 202214942
* Juan Miguel Delgado – 202314903
* Omar Mauricio Urrego Vásquez – 202211641
* Juan Felipe Lancheros Carrillo – 202211004

## 1. Descripción del Problema
### 1.1. Formulación matemática del CVRP
A continuación, se detalla cada elemento de la formulación base, manteniendo las simplificaciones realizadas en la anterior entrega y alineándose con lo solicitado para esta etapa.

* $D = \{CD01\} = \{1\}$: Conjunto unitario que representa el único centro de distribución de la red (índice: $i \in D$).
* $C  = \{C001, C002, ..., Cn\} = \{1, 2, ..., n\}$: Conjunto que representa los clientes de la red (índices: $j, l \in C$).
* $N = D \cup C$: Representa todos los nodos de la red  (índices: $p, q \in D \cup C$)
* $K$: Conjunto de vehículos (índice: $k \in K$) disponibles en la flota: $K = \{V001, V002, ..., Vm\} = \{1, 2, ..., m\}$.
  * Se asume que un vehículo podría distribuir recursos a varios clientes en un mismo viaje si tiene la capacidad requerida.

Nota: Se asume que cada tipo de nodo que conforma la red (el centro de distribución y los clientes) debe representarse como un conjunto por separado, debido a que, en el repositorio, cada uno de ellos tiene identificadores propios (ClientID y DepotID).

### 1.2. Características de la instancia base

### 1.2.1. Parámetros
* $Latitude_p \equiv Latitude_q$ (adimensional): Primer tipo de coordenada geográfica angular de un nodo de la red.
* $Longitude_p \equiv Longitude_q$ (adimensional): Segundo tipo de coordenada geográfica angular de un nodo de la red.
* $LocationID_p \equiv LocationID_q$ (adimensional): Hace referencia a una combinación única de valores correspondientes a los parámetros que representan la latitud y la longitud.
* $Demand_j \equiv Demand_l$ (kg): Cantidad de masa requerida por un cliente.
* $Capacity_k$ (kg): Capacidad máxima de carga de un vehículo.
* $Range_k$ (km): Distancia máxima que recorre un vehículo sin recargar combustible.
* $FuelEfficiency_k$ (km/galón): Eficiencia del uso de combustible de un vehículo.
  * Se toma como referencia el siguiente valor (obtenido del repositorio): 30 km/galón.
* $C^{fuel}$ (COP/galón): Costo del combustible.
  * Se asume que hay un único tipo de combustible.
  * No tiene índice porque tiene un valor fijo en todos los casos.
    * Se toma como referencia el siguiente valor (obtenido del repositorio): 16300 COP/galón.
* $C^{dist}_k$ (COP/km): El valor a pagar por cada kilómetro que recorre un vehículo particular no es considerado en el caso base ya que no hace parte de la estructura básica de costos que debemos considerar, mostrada en el repositorio.
* $C^{maintenance}$ (COP/km): Valor de mantenimiento a pagar basado en la distancia recorrida.
  * No tiene índice porque tiene un valor fijo en todos los casos.
  * Se debe investigar e incorporar al archivo parameters_base.csv del repositorio.
* $C^{time}_k$ (COP/hora): Costo operativo (salario de un conductor) por hora del vehículo $k$.
  * Se debe investigar (TuSalario.org) e incorporar al archivo parameters_base.csv del repositorio.
  * Se asume que es un valor fijo.
* $d_{pq}$ (km): Distancia que conecta un nodo $p$ con un nodo $q$.
  * Sus valores se obtienen del preprocesamiento.
* $t_{pq}$ (horas): Tiempo que toma un vehículo en llegar de un nodo $p$ a un nodo $q$.
  * Sus valores se obtienen del preprocesamiento.
* $C^{fixed}_k$ (COP/vehículo): El costo fijo de activación de un vehículo particular no es considerado en el caso base ya que no hace parte de la estructura básica de costos que debemos considerar, mostrada en el repositorio.
* $VehicleQuantity$: Representa la cantidad de vehículos disponibles.
  * No tiene índice porque tiene un valor fijo en todos los casos.
  * Se calcula al contar la cantidad de vehículos presentes en el archivo csv de entrada.
  * Se asume que todos los vehículos deben ser usados.

### 1.2.2. Variables de decisión
* $x_{pqk} \in \{0, 1\}$: Representa si un vehículo $k$ fue asignado para transportarse entre un nodo $p$ a un nodo $q$..

### 1.2.3. Función objetivo
$$\min{(\sum_{p \in D \cup C}{\sum_{q \in D \cup C}{\sum_{k \in K}{(([A]) * [B]) * x_{pqk}}}})}$$
* $[A]$: $\frac{C^{fuel}}{FuelEfficiency_k} + C^{maintenance}$
* $[B]$: $d_{pq} + C^{time}_k * t_{pq}$

### 1.3. Restricciones y consideraciones
a) Los vehículos deben salir del centro de distribución.
$$\sum_{j \in C}{\sum_{k \in K}{x_{ijk}}} = VehicleQuantity \quad \forall{i \in D}$$
b) Todos los vehículos deben volver al centro de distribución.
$$\sum_{j \in C}{\sum_{k \in K}{x_{ijk}}} - \sum_{j \in C}{\sum_{k \in K}{x_{jik}}} = 0 \quad \forall{i \in D}$$
c) Los clientes deben ser visitados una única vez.
$$\sum_{p \in D \cup C}{\sum_{k \in K}{x_{pjk}}} = 1 \quad \forall{j \in C}$$
d) Cualquier vehículo que llegue/pase por un cliente debe salir de ahí, es decir que ningún vehículo debe terminar su ruta en un cliente.
$$\sum_{p \in D \cup C}{x_{pjk}} - \sum_{p \in D \cup C}{x_{jpk}} = 0 \quad \forall{j \in C} \quad \forall{k \in K}$$
e) El total de distancia que recorre un vehículo en una ruta debe ser menor o igual a su autonomía, es decir que no debe tener el requerimiento de tanquear durante el viaje.
$$\sum_{p \in D \cup C}{\sum_{q \in D \cup C}{x_{pqk} * d_{pq}}} \le Range_k \quad \forall{k \in K}$$
f) La demanda que satisface un vehículo debe ser menor o igual a su capacidad máxima de peso para evitar que colapse.  
$$\sum_{p \in D \cup C}{\sum_{j \in C}{x_{pjk} * Demand_j}} \le Capacity_k \quad \forall{k \in K}$$
g) Un vehículo podría pasar por un trayecto desde un nodo a otro o viceversa. Si pasó por un trayecto específico y vuelve a pasar por el mismo en el sentido contrario, no debería volver a pasar por dicho terreno.
$$x_{pqk} + x_{qpk} \le 2 \quad \forall{p, q \in D \cup C} \quad \forall{k \in K}$$
h) No deben haber trayectorias que no representen desplazamientos significativos.
$$x_{pqk} = 0 \quad \forall{p, q \in D \cup C | p=q} \quad \forall{k \in K}$$


## 2. Método implementado

### 2.1. Descripción detallada del método metaheurístico implementado
### 2.2. Estrategias de representación y operadores
### 2.3. Proceso de calibración de parámetros


## 3. Resultados Experimentales

### 3.1. Configuración experimental


### 3.2. Presentación de resultados por método


### 3.3. Análisis comparativo con soluciones Pyomo


### 3.4. Visualizaciones y gráficos relevantes

## 4. Análisis de Escalabilidad


### 4.1. Rendimiento en instancias de diferentes tamaños


### 4.2. Límites prácticos de aplicabilidad


### 4.3. Estrategias para mejorar la escalabilidad


## 5. Discusión
### 5.1. Ventajas y desventajas de cada enfoque

### 5.2. Recomendaciones para diferentes escenarios

### 5.3. Lecciones aprendidas y desafíos encontrados

## 6. Conclusiones
### 6.1. Resumen de hallazgos principales

### 6.2. Respuestas a preguntas estratégicas

### 6.3. Direcciones futuras de investigación